In [1]:
import os, sys
import io
import skimage.io as skimg
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image
import IPython.display as display

In [2]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _floats_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [3]:
# images and labels array as input
def convert_to(images_name, info):
    filename = '/Users/jakechiu/Desktop/NEU-DET/img.tfrecords'
    #print('Writing', filename)
    writer = tf.io.TFRecordWriter(filename)
    for index in images_name:
        pa = '/Users/jakechiu/Desktop/NEU-DET/image/'
        image_path = pa + index
        image = skimg.imread(image_path)
        image_raw = image.tostring()
        img_info = info.loc[info['filename'] == index]
        h = img_info['height']
        w = img_info['width']
        d = img_info['depth']
        label = img_info['class']
        xmin = img_info['xmin']
        xmax = img_info['xmax']
        ymin = img_info['ymin']
        ymax = img_info['ymax']
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(h),
            'width': _int64_feature(w),
            'depth': _int64_feature(d),
            'image_raw': _bytes_feature(image_raw),
            'label': _int64_feature(label),
            'xmin': _floats_feature(xmin),
            'xmax': _floats_feature(xmax),
            'ymin': _floats_feature(ymin),
            'ymax': _floats_feature(ymax)}))
        writer.write(example.SerializeToString())
    writer.close()

In [4]:
path = '/Users/jakechiu/Desktop/NEU-DET/image/'
img_name = os.listdir(path)
img_name

['scratches_225.jpg',
 'scratches_231.jpg',
 'scratches_219.jpg',
 'patches_162.jpg',
 'pitted_surface_13.jpg',
 'patches_176.jpg',
 'patches_189.jpg',
 'crazing_49.jpg',
 'crazing_8.jpg',
 'patches_22.jpg',
 'crazing_61.jpg',
 'crazing_75.jpg',
 'patches_36.jpg',
 'pitted_surface_228.jpg',
 'inclusion_58.jpg',
 'rolled-in_scale_239.jpg',
 'crazing_219.jpg',
 'pitted_surface_214.jpg',
 'crazing_231.jpg',
 'rolled-in_scale_205.jpg',
 'inclusion_197.jpg',
 'inclusion_64.jpg',
 'inclusion_183.jpg',
 'inclusion_70.jpg',
 'crazing_225.jpg',
 'rolled-in_scale_211.jpg',
 'pitted_surface_200.jpg',
 'inclusion_168.jpg',
 'rolled-in_scale_18.jpg',
 'rolled-in_scale_30.jpg',
 'inclusion_154.jpg',
 'inclusion_140.jpg',
 'rolled-in_scale_24.jpg',
 'patches_228.jpg',
 'scratches_153.jpg',
 'scratches_147.jpg',
 'patches_214.jpg',
 'patches_200.jpg',
 'scratches_190.jpg',
 'scratches_39.jpg',
 'scratches_184.jpg',
 'scratches_11.jpg',
 'crazing_147.jpg',
 'rolled-in_scale_173.jpg',
 'pitted_surface_1

In [5]:
info = pd.read_csv('/Users/jakechiu/Desktop/NEU-DET/Annotation.csv')
info.replace(['crazing', 'inclusion', 'patches', 'pitted_surface', 'rolled-in_scale', 'scratches'], [1, 2, 3, 4, 5, 6], inplace=True)
info['class'].unique()

array([1, 2, 3, 4, 5, 6])

In [6]:
# transform image and label to TFRecord file
path = '/Users/jakechiu/Desktop/NEU-DET/image/'
img_name = os.listdir(path)
info = pd.read_csv('/Users/jakechiu/Desktop/NEU-DET/Annotation.csv')
info.replace(['crazing', 'inclusion', 'patches', 'pitted_surface', 'rolled-in_scale', 'scratches'], [1, 2, 3, 4, 5, 6], inplace=True)
convert_to(img_name, info)


In [7]:
file_path = '/Users/jakechiu/Desktop/NEU-DET/img.tfrecords'
reader = tf.data.TFRecordDataset(file_path)
feature_set = {'height': tf.io.FixedLenFeature([], tf.int64),
               'width': tf.io.FixedLenFeature([], tf.int64),
               'depth': tf.io.FixedLenFeature([], tf.int64),
               'image_raw': tf.io.FixedLenFeature([], tf.string),
               'label': tf.io.FixedLenFeature([], tf.int64),
               'xmin': tf.io.FixedLenFeature([], tf.float32),
               'xmax': tf.io.FixedLenFeature([], tf.float32),
               'ymin': tf.io.FixedLenFeature([], tf.float32),
               'ymax': tf.io.FixedLenFeature([], tf.float32)
              }

In [8]:
def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_set)

In [9]:
parsed_image_dataset = reader.map(_parse_image_function)
parsed_image_dataset

<MapDataset shapes: {depth: (), height: (), image_raw: (), label: (), width: (), xmax: (), xmin: (), ymax: (), ymin: ()}, types: {depth: tf.int64, height: tf.int64, image_raw: tf.string, label: tf.int64, width: tf.int64, xmax: tf.float32, xmin: tf.float32, ymax: tf.float32, ymin: tf.float32}>

In [10]:
for image_features in parsed_image_dataset:
    image_raw = image_features['image_raw'].numpy()
    #print(image_raw)
    #display.display(display.Image(data=image_raw))

InvalidArgumentError: Key: width.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]